### Loading packages

In [1]:
import os
import sys
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sqlite3
from sqlite3 import Error

### Loading [Data from Kaggle competition](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions)

In [2]:
path = './data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

./data/RAW_recipes.csv
./data/interactions_test.csv
./data/interactions_validation.csv
./data/PP_recipes.csv
./data/.~lock.PP_recipes.csv#
./data/selected_recipes.h5
./data/new_basic_features.h5
./data/PP_users.csv
./data/FoodFlix-data.xls
./data/selected_recipes.csv
./data/new_basic_features.csv
./data/interactions_train.csv
./data/RAW_interactions.csv


In [19]:
df_recipes_raw = pd.read_csv('./data/RAW_recipes.csv')
df_recipes_cod = pd.read_csv('./data/PP_recipes.csv')

df_recipes_raw.tags = df_recipes_raw.tags.apply(eval)
df_recipes_raw.ingredients = df_recipes_raw.ingredients.apply(eval)
df_recipes_raw.steps = df_recipes_raw.steps.apply(eval)

### Examining Data

In [20]:
df_recipes_raw.head(3)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13


In [21]:
df_recipes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
name              231636 non-null object
id                231637 non-null int64
minutes           231637 non-null int64
contributor_id    231637 non-null int64
submitted         231637 non-null object
tags              231637 non-null object
nutrition         231637 non-null object
n_steps           231637 non-null int64
steps             231637 non-null object
description       226658 non-null object
ingredients       231637 non-null object
n_ingredients     231637 non-null int64
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [22]:
df_recipes_cod.head(3)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."


In [23]:
df_recipes_cod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178265 entries, 0 to 178264
Data columns (total 8 columns):
id                   178265 non-null int64
i                    178265 non-null int64
name_tokens          178265 non-null object
ingredient_tokens    178265 non-null object
steps_tokens         178265 non-null object
techniques           178265 non-null object
calorie_level        178265 non-null int64
ingredient_ids       178265 non-null object
dtypes: int64(3), object(5)
memory usage: 10.9+ MB


In [24]:
df_recipes_raw[df_recipes_raw.id==424415]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
8586,aromatic basmati rice rice cooker,424415,61,496803,2010-05-10,"[weeknight, time-to-make, course, main-ingredi...","[228.2, 2.0, 2.0, 8.0, 9.0, 1.0, 15.0]",6,"[rinse the rice in a fine strainer , then drai...",from the ultimate rice cooker cookbook. the a...,"[basmati rice, water, salt, cinnamon stick, gr...",5


In [25]:
df_recipes_cod[df_recipes_cod.id==424415]

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"


#### Examining the "Tag" field

In [26]:
df_recipes_raw.tags.head()

0    [60-minutes-or-less, time-to-make, course, mai...
1    [30-minutes-or-less, time-to-make, course, mai...
2    [time-to-make, course, preparation, main-dish,...
3    [60-minutes-or-less, time-to-make, course, mai...
4    [weeknight, time-to-make, course, main-ingredi...
Name: tags, dtype: object

In [28]:
tags = set([])
for taglist in df_recipes_raw.tags:
    for tag in taglist:
        tags.add(tag)
        
print(len(tags))

552


#### Examining the "Ingredients" field

In [32]:
ingredients = {}
i = 0
for ingredient_list in df_recipes_raw.ingredients:
    for ingredient in ingredient_list:
        if ingredient not in ingredients:
            ingredients[ingredient] = i
            i += 1

print(len(ingredients))

14942


In [36]:
def convert(lista):
    convertidos = []
    for elem in lista:
        convertidos.append(ingredients[elem])
    return convertidos

In [37]:
df_recipes_raw['ingredients_ids'] = df_recipes_raw['ingredients'].apply(convert)

In [38]:
df_recipes_raw.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,ingredients_ids
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7,"[0, 1, 2, 3, 4, 5, 6]"
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6,"[7, 8, 9, 10, 11, 12]"
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 6, 23..."
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11,"[25, 26, 27, 28, 29, 5, 30, 6, 31, 32, 33]"
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8,"[34, 35, 36, 6, 31, 37, 38, 39]"


### Exporting an abridged dataframe for similarity calculations

In [39]:
useful = ['id',
          'name', 
          'ingredients', 
          'ingredients_ids', 
          ]
df_recipes_raw[useful].head()

,id,name,ingredients,ingredients_ids
0,137739,arriba baked winter squash mexican style,"[winter squash, mexican seasoning, mixed spice...","[0, 1, 2, 3, 4, 5, 6]"
1,31490,a bit different breakfast pizza,"[prepared pizza crust, sausage patty, eggs, mi...","[7, 8, 9, 10, 11, 12]"
2,112140,all in the kitchen chili,"[ground beef, yellow onions, diced tomatoes, t...","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 6, 23..."
3,59389,alouette potatoes,"[spreadable cheese with garlic and herbs, new ...","[25, 26, 27, 28, 29, 5, 30, 6, 31, 32, 33]"
4,44061,amish tomato ketchup for canning,"[tomato juice, apple cider vinegar, sugar, sal...","[34, 35, 36, 6, 31, 37, 38, 39]"


In [41]:
df_recipes_raw[useful].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 4 columns):
id                 231637 non-null int64
name               231636 non-null object
ingredients        231637 non-null object
ingredients_ids    231637 non-null object
dtypes: int64(1), object(3)
memory usage: 7.1+ MB


In [44]:
df_recipes_raw[useful].to_pickle("basic_features2.pkl.bz2")

### Building a database from the data (TBFinished)

In [62]:
db_file = 'recipes.sqlite'

def create_connection(db_file):
    """ 
    create a database connection to the SQLite database
    specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


def create_table(conn, create_table_sql):
    """ 
    create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [71]:
sql_create_ingredients_table = r""" CREATE TABLE IF NOT EXISTS ingredients (
                               id integer PRIMARY KEY,
                               ingredient_name text NOT NULL
                               ); """
 
sql_create_tags_table = r"""CREATE TABLE IF NOT EXISTS tags (
                        id integer PRIMARY KEY,
                        tag_name text NOT NULL
                        );"""
 
sql_create_recipes_table = r"""CREATE TABLE IF NOT EXISTS recipes (
                           id integer PRIMARY KEY,
                           recipe_name text NOT NULL
                           );"""   

sql_create_recipes_tags = r"""CREATE TABLE IF NOT EXISTS recipes_tags (
                           id integer PRIMARY KEY,
                           recipe_id integer NOT NULL,
                           tag_id integer NOT NULL,
                           FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                           FOREIGN KEY (tag_id) REFERENCES tags (id)
                           );""" 

sql_create_recipes_ingredients = r"""CREATE TABLE IF NOT EXISTS recipes_ingredients (
                                 id integer PRIMARY KEY,
                                 recipe_id integer NOT NULL,
                                 ingredient_id integer NOT NULL,
                                 FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                                 FOREIGN KEY (ingredient_id) REFERENCES ingredients (id)
                                 );""" 

# create a database connection
conn = create_connection(db_file)

# create tables
if conn is not None:
    create_table(conn, sql_create_ingredients_table)
    create_table(conn, sql_create_tags_table)
    create_table(conn, sql_create_recipes_table)
    create_table(conn, sql_create_recipes_tags)
    create_table(conn, sql_create_recipes_ingredients)
else:
    print("Error! cannot create the database connection.")

In [72]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, ingredient in enumerate(ingredients):
    cur.execute('INSERT INTO ingredients VALUES(?,?)',(idx, ingredient))

In [73]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, tag in enumerate(tags):
    cur.execute('INSERT INTO tags VALUES(?,?)',(idx, tag))

In [76]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, name in enumerate(df_recipes.name):
    try:
        cur.execute('INSERT INTO recipes VALUES(?,?)',(idx, name))
    except:
        print(name)

nan


In [77]:
cur.execute("select * from tags limit 10;")
for result in cur.fetchall():
    print(result)